In [15]:
pip install apache-beam

In [16]:
import pandas as pd

# Load the dataset into a DataFrame
df = pd.read_csv('/content/transactions_dataset.csv')

# Display the first few rows of the dataset
df.head()


,Customer ID,Name,Transaction Amount,Transaction Method
0,1,Alex,219.93,Cash
1,2,David,859.56,Credit Card
2,3,Emma,39.98,Credit Card
3,4,John,343.56,Bank Transfer
4,5,David,343.25,Debit Card


In [17]:
pip install apache-beam

In [18]:
import apache_beam as beam

In [27]:
class FilterCashTransactions(beam.DoFn):
    def process(self, element):
        if element['Transaction Method'] != 'Cash':
            yield element

In [32]:
pipeline_options = beam.options.pipeline_options.PipelineOptions()

with beam.Pipeline(options=pipeline_options) as p:
    # Read the CSV file
    transactions = (
        p
        | "Read CSV" >> beam.io.ReadFromText('/content/transactions_dataset.csv', skip_header_lines=1)
        | "Parse CSV" >> beam.Map(lambda line: dict(zip(['Customer ID', 'Name', 'Transaction Amount', 'Transaction Method'], line.split(','))))
        | "Filter Cash Transactions" >> beam.ParDo(FilterCashTransactions())
    )

    # This step is just for demonstration, it writes the result back to a CSV.
    transactions | "Write to CSV" >> beam.io.WriteToText('/content/filtered_transactions.csv')
